In [1]:
from person2vec import data_handler

In [7]:
handler = data_handler.DataHandler()

In [ ]:
handler.get_entity({"name":"Lady Gaga"})

In [ ]:
handler.update_entity(query={"name":"Lady Gaga"}, update_field='occupation', new_value='musician')

In [ ]:
handler.get_entity({"name":"Lady Gaga"})

In [ ]:
handler.entity_count()

In [ ]:
handler.get_entity({"name":"Barack Obama"})

In [ ]:
from person2vec.utils import snippet_creator

In [ ]:
handler.wipe_snippet_collection()

In [ ]:
snippet_creator.snippetize_db(handler)

In [2]:
from person2vec.train_embeddings import train

Using TensorFlow backend.


In [3]:
from person2vec.test_embeddings import test_tasks

In [4]:
from person2vec.generators import training_data_generator
data_gen = training_data_generator.EmbeddingDataGenerator(300,4)

In [ ]:
model = train.train_model(data_gen=data_gen)

In [ ]:
model, data_gen = model

In [ ]:
model.save_weights('../person2vec/data/weights/base_architecture_100epochs.h5')

In [ ]:
model = train._build_default_model()

In [ ]:
model.load_weights('../person2vec/data/weights/base_architecture_100epochs.h5')

In [ ]:
test_tasks.test_model(model, data_gen=data_gen)

In [ ]:
test_tasks.test_word2vec(data_gen.word_vectors, data_gen=data_gen)

In [5]:
from keras.layers import Input, Flatten, Dropout, Embedding, Dense
from keras.models import Model
from keras.layers.merge import Concatenate
from keras.optimizers import adam

In [8]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 400

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model_1 = Model([input_tensor_words, input_tensor_entity], full_out)

model_1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
word_input (InputLayer)          (None, 32, 300)       0                                            
____________________________________________________________________________________________________
entity_input (InputLayer)        (None, 4)             0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 9600)          0           word_input[0][0]                 
____________________________________________________________________________________________________
entity_embedding (Embedding)     (None, 4, 400)        678000      entity_input[0][0]               
___________________________________________________________________________________________

In [9]:
opt = adam()
loss = 'categorical_crossentropy'
model_1.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [13]:
model_1, data_gen = train.train_model(model=model_1, embed_size=400, epochs=149, data_gen=data_gen)

Epoch 1/149
1024/1024 [==============================] - 17s - loss: 0.9397 - acc: 0.5842    
Epoch 2/149
1024/1024 [==============================] - 16s - loss: 0.7759 - acc: 0.6597    
Epoch 3/149
1024/1024 [==============================] - 17s - loss: 0.7293 - acc: 0.6754    
Epoch 4/149
1024/1024 [==============================] - 16s - loss: 0.6996 - acc: 0.6888    
Epoch 5/149
1024/1024 [==============================] - 16s - loss: 0.6745 - acc: 0.6959    
Epoch 6/149
1024/1024 [==============================] - 16s - loss: 0.6728 - acc: 0.6971    
Epoch 7/149
1024/1024 [==============================] - 16s - loss: 0.6628 - acc: 0.7000    
Epoch 8/149
1024/1024 [==============================] - 17s - loss: 0.6550 - acc: 0.7014    
Epoch 9/149
1024/1024 [==============================] - 17s - loss: 0.6482 - acc: 0.7073    
Epoch 10/149
1024/1024 [==============================] - 17s - loss: 0.6423 - acc: 0.7100    
Epoch 11/149
1024/1024 [==============================] - 1

In [14]:
test_tasks.test_model(model_1, embed_size=400, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.2508 - acc: 0.5453 - val_loss: 0.1514 - val_acc: 0.8419
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.1163 - acc: 0.9333 - val_loss: 0.0882 - val_acc: 0.9289
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0709 - acc: 0.9707 - val_loss: 0.0682 - val_acc: 0.9447
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0533 - acc: 0.9747 - val_loss: 0.0581 - val_acc: 0.9447
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0442 - acc: 0.9800 - val_loss: 0.0514 - val_acc: 0.9526
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0382 - acc: 0.9813 - val_loss: 0.0472 - val_acc: 0.9565
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0340 - acc: 0.9800 - val_loss: 0.0443 - val_acc: 0.9526
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0310 - acc: 0.9787 - val_l

In [16]:
model_1.save_weights('../person2vec/data/weights/embed400_150epochs.h5')

In [21]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 500

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [22]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])



In [23]:
model, data_gen = train.train_model(model=model, embed_size=500, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 17s - loss: 1.1791 - acc: 0.4498    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 0.9256 - acc: 0.5892    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.7726 - acc: 0.6595    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7277 - acc: 0.6741    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.6951 - acc: 0.6875    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6837 - acc: 0.6904    
Epoch 7/150
1024/1024 [==============================] - 17s - loss: 0.6764 - acc: 0.6955    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6633 - acc: 0.7003    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6593 - acc: 0.7021    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6517 - acc: 0.7075    
Epoch 11/150
1024/1024 [==============================] - 1

In [24]:
test_tasks.test_model(model, embed_size=500, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1145 - acc: 0.9200 - val_loss: 0.0769 - val_acc: 0.9565
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0555 - acc: 0.9693 - val_loss: 0.0510 - val_acc: 0.9723
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0385 - acc: 0.9773 - val_loss: 0.0418 - val_acc: 0.9723
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0313 - acc: 0.9827 - val_loss: 0.0360 - val_acc: 0.9763
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0268 - acc: 0.9853 - val_loss: 0.0328 - val_acc: 0.9723
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0237 - acc: 0.9853 - val_loss: 0.0297 - val_acc: 0.9763
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0214 - acc: 0.9867 - val_loss: 0.0283 - val_acc: 0.9723
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0197 - acc: 0.9867 - val_l

In [25]:
model.save_weights('../person2vec/data/weights/embed500_100epochs.h5')

In [26]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 600

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [27]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [28]:
model, data_gen = train.train_model(model=model, embed_size=600, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 17s - loss: 1.2156 - acc: 0.4156    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 0.9913 - acc: 0.5566    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.8034 - acc: 0.6498    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7456 - acc: 0.6698    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.7217 - acc: 0.6786    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6907 - acc: 0.6913    
Epoch 7/150
1024/1024 [==============================] - 16s - loss: 0.6739 - acc: 0.6957    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6691 - acc: 0.6980    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6662 - acc: 0.6986    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6529 - acc: 0.7048    
Epoch 11/150
1024/1024 [==============================] - 1

In [29]:
test_tasks.test_model(model, embed_size=600, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1620 - acc: 0.8240 - val_loss: 0.0990 - val_acc: 0.9368
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0654 - acc: 0.9667 - val_loss: 0.0643 - val_acc: 0.9407
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0436 - acc: 0.9747 - val_loss: 0.0528 - val_acc: 0.9526
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0349 - acc: 0.9747 - val_loss: 0.0476 - val_acc: 0.9407
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0297 - acc: 0.9827 - val_loss: 0.0427 - val_acc: 0.9565
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0263 - acc: 0.9813 - val_loss: 0.0406 - val_acc: 0.9526
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0239 - acc: 0.9840 - val_loss: 0.0387 - val_acc: 0.9526
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0218 - acc: 0.9853 - val_l

In [30]:
model.save_weights('../person2vec/data/weights/embed600_150epochs.h5')

In [31]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 700

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [32]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [33]:
model, data_gen = train.train_model(model=model, embed_size=700, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 17s - loss: 1.2275 - acc: 0.4052    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 0.9771 - acc: 0.5634    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.7900 - acc: 0.6553    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7334 - acc: 0.6743    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.7036 - acc: 0.6839    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6849 - acc: 0.6915    
Epoch 7/150
1024/1024 [==============================] - 16s - loss: 0.6743 - acc: 0.6920    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6704 - acc: 0.6961    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6677 - acc: 0.6973    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6538 - acc: 0.7034    
Epoch 11/150
1024/1024 [==============================] - 1

In [34]:
test_tasks.test_model(model, embed_size=700, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1629 - acc: 0.8160 - val_loss: 0.0901 - val_acc: 0.9447
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0573 - acc: 0.9747 - val_loss: 0.0522 - val_acc: 0.9723
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0372 - acc: 0.9813 - val_loss: 0.0410 - val_acc: 0.9763
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0294 - acc: 0.9800 - val_loss: 0.0356 - val_acc: 0.9763
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0250 - acc: 0.9840 - val_loss: 0.0319 - val_acc: 0.9802
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0219 - acc: 0.9880 - val_loss: 0.0290 - val_acc: 0.9802
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0198 - acc: 0.9867 - val_loss: 0.0269 - val_acc: 0.9842
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0180 - acc: 0.9907 - val_l

In [35]:
model.save_weights('../person2vec/data/weights/embed700_150epochs.h5')

In [36]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 800

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [37]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [38]:
model, data_gen = train.train_model(model=model, embed_size=800, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 17s - loss: 1.1827 - acc: 0.4393    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 1.0053 - acc: 0.5389    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.7983 - acc: 0.6489    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7451 - acc: 0.6686    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.7161 - acc: 0.6817    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6985 - acc: 0.6872    
Epoch 7/150
1024/1024 [==============================] - 16s - loss: 0.6834 - acc: 0.6895    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6673 - acc: 0.6991    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6712 - acc: 0.6989    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6608 - acc: 0.6992    
Epoch 11/150
1024/1024 [==============================] - 1

In [39]:
test_tasks.test_model(model, embed_size=800, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1649 - acc: 0.7973 - val_loss: 0.0761 - val_acc: 0.9407
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0584 - acc: 0.9667 - val_loss: 0.0458 - val_acc: 0.9605
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0399 - acc: 0.9747 - val_loss: 0.0390 - val_acc: 0.9605
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0330 - acc: 0.9773 - val_loss: 0.0350 - val_acc: 0.9605
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0288 - acc: 0.9773 - val_loss: 0.0330 - val_acc: 0.9644
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0260 - acc: 0.9787 - val_loss: 0.0316 - val_acc: 0.9644
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0237 - acc: 0.9787 - val_loss: 0.0299 - val_acc: 0.9644
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0219 - acc: 0.9813 - val_l

In [40]:
model.save_weights('../person2vec/data/weights/embed800_150epochs.h5')

In [41]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 900

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [42]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [43]:
model, data_gen = train.train_model(model=model,  embed_size=900, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 19s - loss: 1.1802 - acc: 0.4445    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 0.9646 - acc: 0.5664    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.7779 - acc: 0.6582    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7342 - acc: 0.6708    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.7059 - acc: 0.6838    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6899 - acc: 0.6904    
Epoch 7/150
1024/1024 [==============================] - 16s - loss: 0.6874 - acc: 0.6887    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6722 - acc: 0.6951    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6692 - acc: 0.6971    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6568 - acc: 0.7045    
Epoch 11/150
1024/1024 [==============================] - 1

In [44]:
test_tasks.test_model(model, embed_size=900, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1411 - acc: 0.8680 - val_loss: 0.0705 - val_acc: 0.9565
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0489 - acc: 0.9707 - val_loss: 0.0442 - val_acc: 0.9684
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0344 - acc: 0.9760 - val_loss: 0.0386 - val_acc: 0.9723
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0286 - acc: 0.9800 - val_loss: 0.0349 - val_acc: 0.9723
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0249 - acc: 0.9840 - val_loss: 0.0328 - val_acc: 0.9723
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0222 - acc: 0.9853 - val_loss: 0.0305 - val_acc: 0.9723
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0204 - acc: 0.9867 - val_loss: 0.0290 - val_acc: 0.9723
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0186 - acc: 0.9880 - val_l

In [45]:
model.save_weights('../person2vec/data/weights/embed900_150epochs.h5')

In [46]:
num_total_entities = handler.entity_count()
word_vec_size = 300
snip_size = 32
num_compare_entities = 4
embedding_size = 1000

input_tensor_words = Input(shape=(snip_size, word_vec_size,), dtype='float32', name='word_input')
input_tensor_entity = Input(shape=(num_compare_entities,), dtype='int32', name='entity_input')

word_flatten_layer = Flatten()(input_tensor_words)
word_dropout_layer = Dropout(0.)(word_flatten_layer)

entity_embedding_layer = Embedding(num_total_entities, embedding_size, input_length=num_compare_entities, name='entity_embedding')(input_tensor_entity)
entity_embedding_layer = Flatten()(entity_embedding_layer)
entity_embedding_layer = Dropout(0.)(entity_embedding_layer)

word_branch = Dense(1000, activation="relu", name='dense_sentence_layer')(word_dropout_layer)

joint_embeds = Concatenate(name='joint_embeds')([word_branch, entity_embedding_layer])

nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_entity], full_out)

In [47]:
opt = adam()
loss = 'categorical_crossentropy'
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [48]:
model, data_gen = train.train_model(model=model, embed_size=1000, epochs=150, data_gen=data_gen)

Epoch 1/150
1024/1024 [==============================] - 17s - loss: 1.1762 - acc: 0.4498    
Epoch 2/150
1024/1024 [==============================] - 16s - loss: 0.9133 - acc: 0.6001    
Epoch 3/150
1024/1024 [==============================] - 16s - loss: 0.7812 - acc: 0.6539    
Epoch 4/150
1024/1024 [==============================] - 16s - loss: 0.7333 - acc: 0.6733    
Epoch 5/150
1024/1024 [==============================] - 16s - loss: 0.7081 - acc: 0.6865    
Epoch 6/150
1024/1024 [==============================] - 16s - loss: 0.6826 - acc: 0.6922    
Epoch 7/150
1024/1024 [==============================] - 16s - loss: 0.6815 - acc: 0.6950    
Epoch 8/150
1024/1024 [==============================] - 16s - loss: 0.6693 - acc: 0.6985    
Epoch 9/150
1024/1024 [==============================] - 16s - loss: 0.6675 - acc: 0.6986    
Epoch 10/150
1024/1024 [==============================] - 16s - loss: 0.6573 - acc: 0.7007    
Epoch 11/150
1024/1024 [==============================] - 1

In [49]:
test_tasks.test_model(model, embed_size=1000, data_gen=data_gen)

Train on 750 samples, validate on 253 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.1596 - acc: 0.7960 - val_loss: 0.0694 - val_acc: 0.9565
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.0506 - acc: 0.9627 - val_loss: 0.0438 - val_acc: 0.9644
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0352 - acc: 0.9707 - val_loss: 0.0369 - val_acc: 0.9684
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0294 - acc: 0.9747 - val_loss: 0.0325 - val_acc: 0.9723
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0259 - acc: 0.9773 - val_loss: 0.0297 - val_acc: 0.9684
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0233 - acc: 0.9800 - val_loss: 0.0279 - val_acc: 0.9723
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0214 - acc: 0.9840 - val_loss: 0.0266 - val_acc: 0.9763
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0201 - acc: 0.9827 - val_l

In [50]:
model.save_weights('../person2vec/data/weights/embed1000_150epochs.h5')

In [ ]:
handler.get_entities({'name':'Barack Obama'})

In [ ]:
import pandas

In [ ]:
entities = pandas.DataFrame.from_dict(handler.get_all_entities())
entities.set_index('name', inplace=True)

In [ ]:
name_gender = entities.drop(['texts', '_id', 'description', 'occupation'], axis=1)

In [ ]:
name_gender